### Brandon Thimmesch, IMSE 560 TA


# Wyndor Glass Problem

In [1]:
# checking installation status of CPLEX and DOCPLEX packages
!pip install cplex --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org

!pip install docplex --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org

## Importing Libraries

In [2]:
import sys
import docplex.mp as mp
from docplex.mp.model import Model

## Basic Model

In [3]:
# this model is a recreation of the Wyndor Glass excel model that can be found on Canvas
class LP_problem_basic: # defining LP model formulation as a function
    def __init__(self): # initializing the LP model
        self.model = Model(name = 'Wyndor Glass co. Product-Mix Problem -- BASIC')
    
    # defining decision variables
    def create_variables(self):
        # decision variable that dictates the quantity of door batches to produce
        self.Doors = self.model.continuous_var(name = 'Batches of Door', lb = 0) # lower bound = 0 (built in non-negativity constraint)
        
        # decision variable that dictates the quantity of window batches to produce
        self.Windows = self.model.continuous_var(name = 'Batches of Windows', lb = 0) # lower bound = 0 (built in non-negativity constraint))

    # defining objective function
    def create_objective(self):
        # objective function that maximizes total profits
        self.model.maximize(3000 * self.Doors + 5000 * self.Windows)
        # objective function sums the product of the quantities of door and window batches produced and their respective profit per batch

    # defining constraints
    def create_constraints(self):
        # plant 1 requires 1 hour/batch of doors produced and has 4 hours of production time available
        self.model.add_constraint(self.Doors <= 4) 
        
        # plant 2 requires 2 hours/batch of windows produced and has 12 hours of production time available
        self.model.add_constraint(2 * self.Windows <= 12) 
        
        # plant 3 requires 3 hours/batch of doors produced, 2 hours/batch of windows produced, and has 18 hours of production time available
        self.model.add_constraint(3 * self.Doors + 2 * self.Windows <= 18) 

    # defining function that solves our model when called
    def solve_model(self):
        # solve the optimization problem
        solution = self.model.solve()
        # no need to specify a solver as DOCPLEX will determine which solver is most appropriate for the application
           
        if solution:
            print(self.model.solution)

        else:
            print("No solution found.")
            return None

In [4]:
# calling solver on our model to output a solution
solver = LP_problem_basic()
solver.create_variables()
solver.create_objective()
solver.create_constraints()
solver.solve_model()

solution for: Wyndor Glass co. Product-Mix Problem -- BASIC
objective: 36000
status: OPTIMAL_SOLUTION(2)
Batches of Door=2.000
Batches of Windows=6.000



## Advanced Model
#### Makes use of variables, lists, and arrays for robustness

In [5]:
# defining given constants as variables
door_profit = 3000 # $3,000 profit per batch of doors
window_profit = 5000 # $5,000 profit per batch of windows

# creating matrix to hold production requirements (in hours) for batches of doors and windows at plants 1, 2, and 3
prod_req = [[1,0], # takes 1 hour to produce a batch of doors at plant 1, no window production capacity at plant 1
            [0,2], # # takes 2 hours to produce a batch of windows at plant 2, no door production capacity at plant 2
            [3,2]] # takes 3 hours to produce a batch of doors at plant 3, takes 2 hours to produce a batch of doors at plant 3

# creating list to hold production capacity (in hours) at plants 1, 2, and 3
prod_cap = [4,12,18] # plants 1, 2, and 3 have 4, 12, and 18 production hours available respectively

# this model is a recreation of the Wyndor Glass excel model that can be found on Canvas
class LP_problem_adv: # defining LP model formulation as a function
    def __init__(self): # initializing the LP model
        self.model = Model(name = 'Wyndor Glass co. Product-Mix Problem -- ADVANCED')

    # defining decision variables
    def create_variables(self):
        # decision variable that dictates the quantity of door batches to produce
        self.Doors = self.model.continuous_var(name = 'Batches of Doors', lb = 0) # lower bound = 0 (built in non-negativity constraint)
        
        # decision variable that dictates the quantity of window batches to produce
        self.Windows = self.model.continuous_var(name = 'Batches of Windows', lb = 0) # lower bound = 0 (built in non-negativity constraint)

    # defining objective function
    def create_objective(self):
        # objective function that maximizes total profits
        self.model.maximize(door_profit * self.Doors + window_profit * self.Windows) 
        # objective function sums the product of profit per batch and the quantities of door and window batches produced

    # defining constraints
    def create_constraints(self):
        # plant 1 requires 1 hour/batch of doors produced and has 4 hours of production time available
        self.model.add_constraint(prod_req[0][0] * self.Doors <= prod_cap[0]) 
        
        # plant 2 requires 2 hours/batch of windows produced and has 12 hours of production time available
        self.model.add_constraint(prod_req[1][1] * self.Windows <= prod_cap[1]) 
        
        # plant 3 requires 3 hours/batch of doors produced, 2 hours/batch of windows produced, and has 18 hours of production time available
        self.model.add_constraint(prod_req[2][0] * self.Doors + prod_req[2][1] * self.Windows <= prod_cap[2])     

    # defining function that solves our model when called
    def solve_model(self):
        # solve the optimization problem
        solution = self.model.solve() 
        # no need to specify a solver as DOCPLEX will determine which solver is most appropriate for the application
           
        if solution:
            print(self.model.solution)

        else:
            print("No solution found.")
            return None

In [6]:
# calling solver on our model to output a solution
solver = LP_problem_adv()
solver.create_variables()
solver.create_objective()
solver.create_constraints()
solver.solve_model()

solution for: Wyndor Glass co. Product-Mix Problem -- ADVANCED
objective: 36000
status: OPTIMAL_SOLUTION(2)
Batches of Doors=2.000
Batches of Windows=6.000



We can see that both methods provide results that match the solution found using solver in Excel. You may think the advanced model is overly complicated, but storing your constraint information in a data structure (such as a list, array, or pandas dataframe) will help you create more robust, future-proof models that will be easier to modify in case any provided information changes. Additionally, referencing these arrays by their indices will help you create simplified model formulations that makes use of combinatorial notation such as X_ij (for i belonging to set I, for j belonging to set J).

Please feel free to contact me by email (brandonthimmesch@ksu.edu) if you have any questions.